## Rules
*    The game is based on a population count in an infinite 2-dimensional world.

*   At every iteration (generation) of the world, each cell can either be live or dead.
*   Any live cell with less than 2 members dies, as if by under-population.
*   Any live cell with more than 3 members dies, as if by over-crowding.
*   Ayn dead cell with exactly 3 live members, becomes live, as if by reproduction.

## Grid Design

In [1]:
import pygame
import random

pygame 2.1.2 (SDL 2.0.18, Python 3.10.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()
BLACK = (0,0,0)
GREY = (128,128,128)
YELLOW = (255,255,0)
WHITE = (255,255,255)

width, height = 600,600
tile_size = 20
grid_height = height // tile_size
grid_width = width // tile_size
fps = 60

screen = pygame.display.set_mode((width,height))

clock = pygame.time.Clock()

def draw_grid(positions):
  for position in positions:
    col, row = position
    top_left = (col * tile_size, row *tile_size)
    pygame.draw.rect(screen, YELLOW, (*top_left, tile_size, tile_size))

  for row in range(grid_height):
    pygame.draw.line(screen, BLACK, (0, row * tile_size), (width, row * tile_size))
    #pygame.draw.line(screen, BLACK, (row*tile_size*2, height),(height, row*tile_size*2))
    #pygame.draw.line(screen, BLACK, (-row*tile_size*2, height),(height, -row*tile_size*2))
  for col in range(grid_width):
    pygame.draw.line(screen, BLACK, (col * tile_size, 0), (col * tile_size, height))
    #pygame.draw.line(screen, BLACK, (0,-col*tile_size*2), (height+col*tile_size*2, height))
    #pygame.draw.line(screen, BLACK, (0,col*tile_size*2), (height-col*tile_size*2, height))
    

def gen(number):
  return set([(random.randrange(0,grid_height), random.randrange(0, grid_width))  for _ in range(number)])

## Grid Update

In [3]:
def get_neighbors(pos):
    x,y = pos
    neighbors = []
    for dx in [-1, 0, 1]:
        if x+dx <0 or x+dx > grid_width:
            continue
        for dy in [-1, 0, 1]:
            if y+dy <0 or y+dy > grid_height:
                continue
            if dx == 0 and dy == 0:
                continue
            neighbors.append((x+dx, y+dy))
    
    return neighbors

def adjust_grid(positions):
    all_neighbors = set()
    new_positions = set()

    for posiiton in positions:
        neighbors = get_neighbors(posiiton)
        all_neighbors.update(neighbors)

        neighbors = list(filter(lambda x: x in positions, neighbors))

        if len(neighbors) in [2,3]:
            new_positions.add(posiiton)
    
    for position in all_neighbors:
        neighbors = get_neighbors(position)
        neighbors = list(filter(lambda x: x in positions, neighbors))

        if len(neighbors) == 3:
            new_positions.add(position)
       
    return new_positions

## Main Loop

In [4]:
def main():
    running = True
    playing = True
    time_step = 0
    update_freq = 10

    positions = set()
    while running:
        clock.tick(fps)

        screen.fill(GREY)
        draw_grid(positions)
        pygame.display.update()

        if playing:
            time_step += 1
        
        if time_step >= update_freq:
            time_step = 0
            positions = adjust_grid(positions = positions)

        pygame.display.set_caption("Playing" if playing else "Paused")

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            
            if event.type == pygame.MOUSEBUTTONDOWN:
                x,y = pygame.mouse.get_pos()
                col = x // tile_size
                row = y // tile_size
                pos = (col,row)

                if pos in positions:
                    positions.remove(pos)
                else:
                    positions.add(pos)
            
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    playing = not playing
                
                if event.key == pygame.K_c:
                    positions = set()
                    playing = False
                    time_step = 0
                
                if event.key ==  pygame.K_g:
                    positions = gen(random.randrange(2,5)*grid_width)
    pygame.quit()

if __name__ == "__main__":
    main()